In [1]:
import pandas as pd
import numpy as np

In [3]:
books_data = pd.read_csv('Books.csv')
ratings_data = pd.read_csv('Ratings.csv')
user_data = pd.read_csv('Users.csv')

C:\Users\chery\AppData\Local\Temp\ipykernel_8988\3460514963.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_data = pd.read_csv('Books.csv')


In [4]:
books_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
ratings_data.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
user_data.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
print(books_data.shape)
print(ratings_data.shape)
print(user_data.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [9]:
books_data.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [10]:
user_data.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [11]:
ratings_data.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [12]:
print(books_data.duplicated().sum())
print(ratings_data.duplicated().sum())
print(user_data.duplicated().sum())

0
0
0


Popularity Based Recommender System

In [ ]:
# Merging the ratings dataset with the books dataset on the common column 'ISBN' to create another dataset 
# that includes both ratings and the corresponding book information
ratings_with_name = ratings.merge(books_data,on='ISBN')

In [ ]:
# Group the merged dataset by 'Book-Title' and count the number of ratings for each book 
# to understand how popular a book is by looking at how many times it has been rated
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()

# Renaming the 'Book-Rating' column to 'num_ratings' for clarity
# This makes it easier to interpret that the column contains the number of ratings each book has received
num_rating_df.rename(columns={'Book-Rating':'num_ratings'}, inplace=True)

# Display that the number of ratings has been counted and renamed correctly
num_rating_df.head()

In [ ]:
# Group the merged dataset by 'Book-Title' and calculate the average rating for each book
# Determine how well-rated each book is on average, which helps in recommending highly-rated books
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()

# Rename the 'Book-Rating' column to 'avg_rating' to make clear that the column now contains the average rating for each book, not individual ratings
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'}, inplace=True)

# Display the first few rows of the dataframe to verify that the average ratings were calculated correctly
avg_rating_df.head()

In [ ]:
# Merge the 'num_rating_df' and 'avg_rating_df' dataframes on 'Book-Title'
# Combine the two pieces of information: the number of ratings and the average rating for each book
# This merged dataframe will allow us to recommend books that are both popular (high number of ratings) and well-rated (high average rating)
popular_df = num_rating_df.merge(avg_rating_df, on='Book-Title')

# Display the first few rows to check if the number of ratings and average ratings have been combined correctly
popular_df.head()

In [ ]:
# Filtering the 'popular_df' to include only books that have at least 250 ratings
# Ensure that the recommendations are based on books with a significant number of ratings, making them more reliable and widely liked
# Select top 50 books based on this ranking that limits the recommendations to a manageable number of top-rated popular books
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

# Display the first few rows of the resulting dataframe to ensure the filtering and sorting worked correctly
popular_df.head()

In [ ]:
# Merge the 'popular_df' with the original 'books_data' on 'Book-Title'
# Add additional book details (such as author, publisher, and year) to the previously created 'popular_df'
# Make the recommendations more informative and useful, as users can see more than just the title and rating
popular_df = popular_df.merge(books_data, on='Book-Title')

# Display the first row of the updated dataframe to verify that 'books_data' have been successfully merged
popular_df.head(1)

In [ ]:
# Check the shape of the 'popular_df' dataframe
# Confirm the size of the dataset after merging with 'books_data'
popular_df.shape

# Count the number of duplicate entries in the 'Book-Title' column of 'popular_df'
# Ensure that each book title appears only once in the recommendations
print(popular_df.duplicated('Book-Title').sum())

In [ ]:
# Drop duplicate entries in the 'Book-Title' column from 'popular_df'
# Ensure that each book title appears only once, preventing skewed recommendations due to repeated entries
# Use 'inplace=True' argument to modify the original dataframe directly, rather than creating a copy
popular_df.drop_duplicates('Book-Title', inplace=True)

# Check the shape of the 'popular_df' dataframe again to confirm that the number of rows has been reduced accordingly
print(popular_df.shape)

# Printing the column names of 'popular_df' to verify that the dataframe contains the expected columns 
print(popular_df.columns)

In [ ]:
# Select only the relevant columns ('Book-Title', 'Book-Author', 'Image-URL-M') from 'popular_df'
# Simplify the dataframe to include only the information necessary for making recommendations
# Make the output cleaner and more user-friendly, highlighting the book title, author, and image
popular_df = popular_df[['Book-Title', 'Book-Author', 'Image-URL-M']]

# Displaying the first row of the final 'popular_df' dataframe to verify that it contains only the selected columns
# Ensure that the structure is correct and ready for presentation or further use in the recommendation system
print(popular_df.head(1))

Collaborative Filtering Based Recommender System

In [ ]:
# Group the 'ratings_with_name' dataset by 'User-ID' and counting the number of ratings each user has given
# Identify users who are active enough in rating books, which helps ensure the quality of recommendations
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200

# Extracting the indices of users who have rated more than 200 books
# Retain only those users whose preferences are well-established, improving the reliability 
users_rating_more_than_200 = x[x].index

# Filter the original 'ratings_with_name' dataset to include only ratings from the selected active users
# Reduce the dataset to a more manageable size and focuses on users who contribute significantly to the ratings
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(users_rating_more_than_200)]

# Group the filtered dataset by 'Book-Title' and counting the number of ratings for each book
# Identify books that have received a sufficient number of ratings, which is crucial for making accurate recommendations
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50

# Extract the indices of books that have been rated 50 times or more
# Ensure that only popular books are included in the recommendations, enhancing the overall quality 
book_rating_more_than_49 = y[y].index

In [ ]:
# Filter the 'filtered_rating' dataframe to include only those ratings for books that have been rated 50 times or more
# Ensure that the final dataset contains only popular books, which improves the reliability 
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(book_rating_more_than_49)]

# Print the 'final_ratings' dataframe to verify its contents
# Confirm that the filtering was successful and that the dataset is ready for the next steps 
print(final_ratings)

In [ ]:
# Creating a pivot table from the 'final_ratings' dataframe
# The pivot table has 'Book-Title' as the index and 'User-ID' as the columns, with 'Book-Rating' as the values
# Transform the data into a matrix format where each row represents a book, each column represents a user, 
# and each cell contains the rating given by that user for that book
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

# Display the first few rows of the pivot table to verify its structure and contents
# Ensure that the data is correctly organized for subsequent analysis and modeling 
pt.head()

In [ ]:
# Checking for missing values in the pivot table 'pt'
# Identify any gaps in the data, which may affect the performance of the collaborative filtering algorithm
pt.isnull().sum()

In [ ]:
# Filling missing values in the pivot table 'pt' with 0
pt.fillna(0, inplace=True)

# Display the first few rows of the updated pivot table to verify that the missing values have been filled correctly
pt.head()

In [ ]:
# Import the 'cosine_similarity' function from the 'sklearn.metrics.pairwise' module
# This function computes the cosine similarity between the rows of the provided matrix, 
# it will help determine how similar the ratings patterns of different books are
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity scores for the pivot table 'pt'
# This results in a matrix where each element (i, j) represents the similarity score between book i and book j
similarity_scores = cosine_similarity(pt)

# Print the similarity scores for the first book in the pivot table,
# and provide insight into how similar the first book is to all other books based on user ratings
print(similarity_scores[0])

In [ ]:
# Check the shape of the 'similarity_scores' matrix
# This will return the dimensions of the matrix, which should be (number of books, number of books)
similarity_scores.shape

In [ ]:
# Defining a function that will recommend similar books based on the cosine similarity scores computed earlier
def recommend(book_name):
    # Find the index of the input book title in the pt's index to retrieve the position of the specified book
    index = np.where(pt.index == book_name)[0][0]
    
    # Sort the similarity scores for the specified book in descending order
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]
    
    # Initialize an empty list to store details of the recommended books
    recommended_books = []
    
    # Iterate over the top similar items to gather details for each recommended book
    for i in similar_items:
        # Extract the details of the recommended book from the 'books_data' dataframe using the index
        book_details = books_data[books_data['Book-Title'] == pt.index[i[0]]]
        
        # Append the first (and only) row of the book details to the recommended_books list
        recommended_books.append(book_details.iloc[0])
    
    # Create a new DataFrame from the list of recommended books
    recommended_df = pd.DataFrame(recommended_books)
    
    # Drop any duplicate book titles from the recommendations
    # Ensure that each recommended book appears only once in the final output
    recommended_df.drop_duplicates('Book-Title', inplace=True)
    
    # Select only the relevant columns for the final output
    recommended_df = recommended_df[['Book-Title', 'Book-Author', 'Image-URL-M']]
    
    # Optional: Convert the recommended DataFrame to JSON format for easier handling in web applications
    # recommended_json = recommended_df.to_json(orient='records')
    
    # Return the DataFrame containing the recommended books
    return recommended_df

In [ ]:
# Call the function with specific book title
recommend('Harry Potter and the Prisoner of Azkaban (Book 3)')

In [ ]:
# Call the function with specific year
recommend('1984')

In [ ]:
# Import 'pickle' module to enable serialization of Python objects
import pickle

# Save the pivot table 'pt' to a file named 'pt.pkl' in binary write mode
# Ensure that the preprocessed data is stored and can be reused without recalculating
pickle.dump(pt, open('pt.pkl', 'wb'))

# Save the 'books' DataFrame to a file named 'books.pkl' in binary write mode to preserve the book data
pickle.dump(books, open('books.pkl', 'wb'))

# Save'similarity_scores' matrix to a file named 'similarity_scores.pkl' in binary write mode
# Store the similarity scores for quick access to the precomputed values
pickle.dump(similarity_scores, open('similarity_scores.pkl', 'wb'))

# Save the 'popular_df' DataFrame to a file named 'popular_df.pkl' in binary write mode
# Ensures that the popular books data is preserved for easy access and display in future applications or analyses
pickle.dump(popular_df, open('popular_df.pkl', 'wb'))
